# Running experiments from .yaml and saving results in .csv

In [1]:
import yaml
import csv
import torch
import numpy as np

In [2]:
import torch.nn.functional as F
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(128, 128)
        self.conv2 = GCNConv(128, 16)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)

        return F.log_softmax(x, dim=1)

In [3]:
from measures import Result


device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
torch.set_printoptions(precision=2)
np.set_printoptions(precision=2)

with open('experiments.yaml', 'r') as yaml_file:
    experiments = yaml.safe_load(yaml_file)

with open('results.csv', 'w', newline='') as csv_file:
    fieldnames = ['Dataset', 'AP', 'AF']
    writer = csv.DictWriter(csv_file, fieldnames=fieldnames)
    writer.writeheader()

    for experiment in experiments['experiments']:
        print(experiment['name'], experiment['directory'])
        for i in range(10):
            dataset = torch.load(experiment['directory'] + experiment['name'] + f'{i}.pt')
            result = Result(dataset, GCN().to(device), [dataset.t == t for t in dataset.t.unique()])
            result.learn()
            ap = result.get_average_accuracy()
            af = 0 if experiment['name'] == 'zero_' else result.get_average_forgetting_measure()
            writer.writerow({'Dataset': f'{experiment["name"]}', 'AP': f'{ap}', 'AF': f'{af}'})
            print(f'AP: {ap:.2f}'.ljust(10) + f' AF: {af:.2f}')
        print()

cuda
base_ ./csbm_base/
AP: 0.49   AF: 0.11
AP: 0.47   AF: 0.09

zero_ ./csbm_zero/
AP: 0.28   AF: 0.00
AP: 0.34   AF: 0.00

feat_ ./csbm_feat/
AP: 0.27   AF: 0.24
AP: 0.31   AF: 0.17

struct_ ./csbm_struct/
AP: 0.25   AF: 0.07
AP: 0.25   AF: 0.06

hom_ ./csbm_hom/
AP: 0.46   AF: 0.15
AP: 0.49   AF: 0.11

class_ ./csbm_class/
AP: 0.56   AF: 0.06
AP: 0.53   AF: 0.07



In [4]:
import pandas as pd

pd.set_option('display.float_format', lambda x: f'{x:.3f}')
performance = pd.read_csv('results.csv')
print(performance.groupby('Dataset').mean())

           AP    AF
Dataset            
base_   0.480 0.103
class_  0.547 0.064
feat_   0.288 0.206
hom_    0.475 0.128
struct_ 0.250 0.064
zero_   0.312 0.000


C:\Users\Produktivität\AppData\Local\Temp\ipykernel_4488\2627822054.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
